In [1]:
import torch as t
from typing import Union
from torch import nn
import torch.nn.functional as F
import plotly.express as px
import plotly.graph_objects as go
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange
from fancy_einsum import einsum
import os
from tqdm.auto import tqdm
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import wandb
import w5d1_utils

/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: <D9493EF5-8DAB-3A5D-85D5-684F04544B84> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <52337463-FA04-3610-8715-68C586545714> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from GAN_implementation.models import Generator, Discriminator
import w5d1_utils



In [3]:
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder

image_size = 64

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = ImageFolder(
    root="img_align_celeba",
    transform=transform
)

w5d1_utils.show_images(trainset, rows=3, cols=10)

In [4]:
discriminator = Discriminator(img_size=64, img_channels=3, generator_num_features=1024, n_layers=4)
generator = Generator(latent_dim_size = 128, img_size=64, img_channels=3, generator_num_features=1024, n_layers=4)


In [5]:
import plotly.express as px
px.imshow(trainset[0][0].permute(1,2,0))

In [6]:
discriminator(trainset[0][0].unsqueeze(0)) # discriminator works

tensor([[0.5063]], grad_fn=<SigmoidBackward0>)

In [7]:
import torch as t
generator(t.randn(1,128)).shape # generator works
px.imshow(generator(t.randn(1,128)).detach().squeeze(0).permute(1,2,0))

In [1]:
from w5d1_solutions import celeb_DCGAN
import w5d1_utils
from GAN_implementation.models import Generator
my_Generator = Generator(100, 63, 3, 1024, 4)
w5d1_utils.print_param_count(my_Generator, celeb_DCGAN.netG)

/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: <D9493EF5-8DAB-3A5D-85D5-684F04544B84> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <52337463-FA04-3610-8715-68C586545714> /Users/josephbloom/miniforge3/envs/arena/lib/python3.10/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


Model 1, total params = 12658432
Model 2, total params = 12658432
All parameter counts match!


,name_1,shape_1,num_params_1,num_params_2,shape_2,name_2
0,project_and_reshape.0.weight,"(16384, 100)",1638400,1638400,"(16384, 100)",project_and_reshape.0.weight
1,project_and_reshape.2.weight,"(1024,)",1024,1024,"(1024,)",project_and_reshape.2.weight
2,project_and_reshape.2.bias,"(1024,)",1024,1024,"(1024,)",project_and_reshape.2.bias
3,layers.0.0.weight,"(1024, 512, 4, 4)",8388608,8388608,"(1024, 512, 4, 4)",layers.0.0.weight
4,layers.0.1.weight,"(512,)",512,512,"(512,)",layers.0.1.weight
5,layers.0.1.bias,"(512,)",512,512,"(512,)",layers.0.1.bias
6,layers.1.0.weight,"(512, 256, 4, 4)",2097152,2097152,"(512, 256, 4, 4)",layers.1.0.weight
7,layers.1.1.weight,"(256,)",256,256,"(256,)",layers.1.1.weight
8,layers.1.1.bias,"(256,)",256,256,"(256,)",layers.1.1.bias
9,layers.2.0.weight,"(256, 128, 4, 4)",524288,524288,"(256, 128, 4, 4)",layers.2.0.weight


In [73]:
from GAN_implementation.models import Discriminator
my_Discriminator = Discriminator(64, 3, 512, 4)
w5d1_utils.print_param_count(my_Discriminator, celeb_DCGAN.netD)

Model 1, total params = 2766528
Model 2, total params = 11038080
Parameter counts don't match up exactly.


,name_1,shape_1,num_params_1,num_params_2,shape_2,name_2
0,layers.0.0.weight,"(64, 3, 4, 4)",3072,6144,"(128, 3, 4, 4)",layers.0.0.weight
1,layers.0.0.bias,"(64,)",64,128,"(128,)",layers.0.0.bias
2,layers.1.0.weight,"(128, 64, 4, 4)",131072,524288,"(256, 128, 4, 4)",layers.1.0.weight
3,layers.1.0.bias,"(128,)",128,256,"(256,)",layers.1.0.bias
4,layers.1.1.weight,"(128,)",128,256,"(256,)",layers.1.1.weight
5,layers.1.1.bias,"(128,)",128,256,"(256,)",layers.1.1.bias
6,layers.2.0.weight,"(256, 128, 4, 4)",524288,2097152,"(512, 256, 4, 4)",layers.2.0.weight
7,layers.2.0.bias,"(256,)",256,512,"(512,)",layers.2.0.bias
8,layers.2.1.weight,"(256,)",256,512,"(512,)",layers.2.1.weight
9,layers.2.1.bias,"(256,)",256,512,"(512,)",layers.2.1.bias


In [6]:
import torch.nn as nn
def initialize_weights_discriminator(model) -> None:
    for name, param in model.named_parameters():
        # print(name)
        if "1.weight" in name: # batchnorm wieght
            nn.init.normal_(param, 1, 0.02)
        elif "1.bias" in name: # batchnorm bias
            nn.init.constant_(param, 0)
        else: # anything else
            nn.init.normal_(param, 0, 0.02)

initialize_weights_discriminator(my_Discriminator)

In [7]:
import torch.nn as nn
def initialize_weights_generator(model) -> None:
    for name, param in model.named_parameters():
        # print(name)
        if (len(param.shape) == 1) and ("project" not in name): # batch norms (and project and reshape)
            if "weight" in name:
                nn.init.normal_(param, 1, 0.02)
            else:
                assert "bias" in name 
                nn.init.constant_(param, 0)
        else:
            nn.init.normal_(param, 0, 0.02)

initialize_weights_generator(my_Generator)

In [ ]:
# Ok now we're going to write our GAN training loop 
# In order of preference I would like the following outcomes:
# 1. Working code
# 2. Well factored code
# 3. Weights and Biases reporting (all important metrics, recording images periodically)
# 4. Well optimized code (work out optimal batch size/other hyper parameters)

In [ ]:
# Pseudo code

# in train
# set up optimizer, loss,

# for epoch in epochs:
#   start with disciminator:
#   generate

In [37]:
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
image_size = (64,64)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = ImageFolder(
    root="img_align_celeba",
    transform=transform
)



In [75]:
from typing import Optional, Union
from torch import optim
import time 
import torch as t
from tqdm.notebook import tqdm
import wandb

def train_generator_discriminator(
    netG: Generator, 
    netD: Discriminator, 
    optG,
    optD,
    trainloader,
    epochs: int,
    max_epoch_duration: Optional[Union[int, float]] = None,           # Each epoch terminates after this many seconds
    log_netG_output_interval: Optional[Union[int, float]] = None,     # Generator output is logged at this frequency
    use_wandb: bool = True,
    device: str = "cpu"):

    criterion = nn.BCELoss()
    fixed_noise = t.randn(64, 100, device=device)

    start_time = time.time()

    if use_wandb:
        wandb.init(project="GAN_implementation", entity="arena-ldn")
        wandb.watch(netG, log="all")
        wandb.watch(netD, log="all")

    
    for epoch in range(epochs):
        pbar = tqdm(enumerate(trainloader, 0), total=len(trainloader))
        for i, data in pbar:
            # 1. Train the discriminator on real+fake
            netD.zero_grad()
            # 1A. Train on real
            real_images = data[0]
            batch_size = real_images.size(0)
            label = t.full((batch_size,), 1, dtype=t.float32)
            output = netD(real_images)
            errD_real = criterion(output.flatten(), label)
            errD_real.backward()
            D_x = output.mean().item()

            # 1B. Train on fake
            noise = t.randn(batch_size, 100, device=device)
            fake = netG(noise)
            label.fill_(0)
            output = netD(fake.detach())
            errD_fake = criterion(output.flatten(), label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optD.step()

            # 2. Train the generator on the discriminator's error
            netG.zero_grad()
            label.fill_(1)
            output = netD(fake)
            errG = criterion(output.flatten(), label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optG.step()

            # 3. Log metrics
            pbar.set_description(f"Epoch: {epoch}, Iteration: {i}, D(x): {D_x:.2f}, D(G(z)): {D_G_z1:.2f}/{D_G_z2:.2f}, D loss: {errD.item():.2f}, G loss: {errG.item():.2f}")

            if i % log_netG_output_interval == 0:
                # print(f"Epoch: {epoch}, Iteration: {i}, D(x): {D_x:.2f}, D(G(z)): {D_G_z1:.2f}/{D_G_z2:.2f}, D loss: {errD.item():.2f}, G loss: {errG.item():.2f}")
                if use_wandb:
                    wandb.log({
                        "D(x)": D_x,
                        "D(G(z))": D_G_z1,
                        "D(G(z))": D_G_z2,
                        "D loss": errD.item(),
                        "G loss": errG.item()
                    })
            
            if use_wandb and (i % 100 == 0):
                with t.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                wandb.log({"images": [wandb.Image(x, caption="epoch: " + str(epoch) + " iteration: " + str(i)) for x in fake]})
            
            if max_epoch_duration is not None:
                if time.time() - start_time > max_epoch_duration:
                    break

        print("Training complete")

    return netG, netD

from GAN_implementation.models import Generator
my_Generator = Generator(100, 64, 3, 512, 4)
initialize_weights_generator(my_Generator)
my_Discriminator = Discriminator(64, 3, 512, 4)
initialize_weights_discriminator(my_Discriminator)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
discriminator_optimizer = optim.Adam(my_Discriminator.parameters(), lr=0.0002, betas=(0.5,0.999))
generator_optimizer = optim.Adam(my_Generator.parameters(), lr=0.0002, betas=(0.5,0.999))

my_Generator, my_Discriminator = train_generator_discriminator(
    my_Generator,
    my_Discriminator,
    generator_optimizer,
    discriminator_optimizer,
    trainloader,
    epochs=1,
    max_epoch_duration=60*60*2,
    log_netG_output_interval=100,
    use_wandb=False,
    device="cpu",
)
    

  0%|          | 0/12663 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [77]:
my_Generator.latent_dim_size

100

In [64]:
my_Generator(t.randn(100, 100)).shape

torch.Size([100, 3, 64, 64])

date and time: 12/06/2022, 12:12:35


In [81]:
noise = t.randn(100, 100)
real_images = next(iter(trainloader))[0]
D_x = my_Discriminator(real_images).mean()
fake = my_Generator(noise)
D_G_z = my_Discriminator(fake.detach()).mean()

(t.log(D_x).mean() + t.log(1 - D_G_z).mean())

tensor(-7.0334e-06, grad_fn=<AddBackward0>)

In [83]:
#Binary Cross Entropy Loss
import torch
torch.nn.BCELoss()(D_x,)

TypeError: BCELoss.forward() missing 1 required positional argument: 'target'

In [84]:
trainloader.batch_size


16